<a href="https://colab.research.google.com/github/felipeturing/pytorch/blob/turing/Define_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEFINING A NEURAL NETWORK IN PYTORCH

In PyTorch, neural networks can be constructed using the `torch.nn` package. It's necessary must install torch with `pip install torch`.

### Imports

An `nn.Module` contain layers, and a method forward  (input) that returns the output.

In [44]:
import torch
import torch.nn as nn 
import torch.nn.functional as F # to manipulate the forward func. between layers

### Define, initiliaze, pass forward the neural network

For this notebook we're gonna built a convolution network. The main features are

1. Extract certain features like edges detection, sharpness, blurriness, etc.
2. Use an small matrix as a kernel, i.e each element of an image is in its local neighbors (convolution operation)

Using convolution, we will define our model to take 1 input image channel, and output match our target of 10 labels representing numbers 0 through 9. We will follow a standard **MNIST** algorithm.

**Important** : An `__init__` function that references `nn.Module` is where you define the fully connected layers in your neural network.

We're gonna use CONV2D class, see [reference](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html?highlight=nn%20conv2d#torch.nn.Conv2d).

`torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)`

A color image is seen as a rectangular box whose width and height are measured by the number of pixels from those dimensions. The three layers of colours (RGB) interpreted by CNNs are referred to as channels.

In [45]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)

        #Designed to ensure that adjacent pixels are either all 0s or all active
        #with an input probability
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)

        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x): # x is the input
        x = self.conv1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = F.relu(x)

        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        
        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)

        output = F.log_softmax(x, dim = 1)
        return output

### Pass data through your model to test

In [46]:
input = torch.rand((1, 1, 28, 28)) #28x28x1 -> 1 channel (The MNIST)grayscale
my_net = Net()
output = my_net(input)
# print(("Input:\n {}").format(input))
print(("Labels predict:\n {}").format(output))

Labels predict:
 tensor([[-2.2877, -2.3330, -2.2935, -2.2946, -2.3128, -2.2188, -2.4090, -2.2306,
         -2.2973, -2.3628]], grad_fn=<LogSoftmaxBackward>)


Finally, each number in this resulting tensor equates to the prediction of the 10 labels the input tensor is associated to.

